In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import *

import matplotlib.pyplot as plt

import sys
import os

import numpy as np

from scipy.misc import imread,imresize

Using TensorFlow backend.


In [18]:
img_dir = "1/"
img_list = os.listdir(img_dir)

data = []
for image_item in img_list:
    data.append(imread(img_dir+image_item)/255.)

data = np.array(data)

In [30]:
class GAN():
    def __init__(self):
        self.img_rows = 96 
        self.img_cols = 96
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        optimizer = SGD(lr=3e-4, momentum=0.9, decay=1e-6, nesterov=True)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', 
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build and compile the generator
        self.generator = self.build_generator()
        self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)

        # The generator takes noise as input and generated imgs
        z = Input(shape=(100,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator) takes
        # noise as input => generates images => determines validity 
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        noise_shape = (100,)
        
        model = Sequential()

        model.add(Dense(256, input_shape=noise_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=noise_shape)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        img_shape = (self.img_rows, self.img_cols, self.channels)
        
        model = Sequential()

        model.add(Flatten(input_shape=img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self,data, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        X_train = data

        # Rescale -1 to 1
#         X_train = (X_train.astype(np.float32) - 127.5) / 127.5
#         X_train = np.expand_dims(X_train, axis=3)

        half_batch = int(batch_size / 2)

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (half_batch, 100))

            # Generate a half batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, 100))

            # The generator wants the discriminator to label the generated samples
            # as valid (ones)
            valid_y = np.array([1] * batch_size)

            # Train the generator
            g_loss = self.combined.train_on_batch(noise, valid_y)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, 100))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = gen_imgs*0.5 + 0.5
#         print(gen_imgs)

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,:])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/mnist_%d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    gan = GAN()
    gan.train(data=data,epochs=30000, batch_size=32, save_interval=200)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_11 (Flatten)         (None, 27648)             0         
_________________________________________________________________
dense_71 (Dense)             (None, 512)               14156288  
_________________________________________________________________
leaky_re_lu_51 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_72 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_52 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_73 (Dense)             (None, 1)                 257       
Total params: 14,287,873
Trainable params: 14,287,873
Non-trainable params: 0
________________________________________________________________

96 [D loss: 0.191845, acc.: 100.00%] [G loss: 1.193850]
97 [D loss: 0.186817, acc.: 100.00%] [G loss: 1.218528]
98 [D loss: 0.158768, acc.: 100.00%] [G loss: 1.195864]
99 [D loss: 0.183891, acc.: 100.00%] [G loss: 1.215914]
100 [D loss: 0.180812, acc.: 100.00%] [G loss: 1.226646]
101 [D loss: 0.178444, acc.: 100.00%] [G loss: 1.224663]
102 [D loss: 0.155359, acc.: 100.00%] [G loss: 1.218086]
103 [D loss: 0.168081, acc.: 100.00%] [G loss: 1.210349]
104 [D loss: 0.161406, acc.: 100.00%] [G loss: 1.251261]
105 [D loss: 0.153344, acc.: 100.00%] [G loss: 1.206990]
106 [D loss: 0.158372, acc.: 100.00%] [G loss: 1.222871]
107 [D loss: 0.152286, acc.: 100.00%] [G loss: 1.249044]
108 [D loss: 0.174860, acc.: 100.00%] [G loss: 1.260534]
109 [D loss: 0.168592, acc.: 100.00%] [G loss: 1.241065]
110 [D loss: 0.158168, acc.: 100.00%] [G loss: 1.256735]
111 [D loss: 0.179204, acc.: 100.00%] [G loss: 1.240166]
112 [D loss: 0.168845, acc.: 100.00%] [G loss: 1.269057]
113 [D loss: 0.179691, acc.: 100.00

KeyboardInterrupt: 

In [19]:
import cv2
cv2.imshow("hehe",(data[1]*255).astype("uint8"))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
cv2.imshow("hehe",data[1])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [17]:
(data[1]*255).astype("uint8")

array([[[106, 117, 139],
        [113, 124, 146],
        [139, 147, 170],
        ..., 
        [ 98, 100, 121],
        [116, 118, 139],
        [142, 143, 164]],

       [[118, 129, 151],
        [150, 158, 181],
        [147, 155, 178],
        ..., 
        [113, 115, 138],
        [122, 124, 145],
        [129, 131, 152]],

       [[122, 129, 155],
        [151, 159, 182],
        [141, 149, 172],
        ..., 
        [138, 140, 163],
        [122, 124, 147],
        [106, 108, 129]],

       ..., 
       [[119, 124, 146],
        [ 98, 103, 125],
        [106, 111, 133],
        ..., 
        [114, 119, 141],
        [118, 123, 145],
        [121, 126, 148]],

       [[106, 111, 133],
        [ 91,  96, 118],
        [101, 106, 128],
        ..., 
        [111, 116, 138],
        [104, 109, 131],
        [106, 111, 133]],

       [[ 86,  91, 113],
        [ 92,  97, 119],
        [102, 107, 129],
        ..., 
        [116, 121, 143],
        [108, 113, 135],
        [133, 138,

In [16]:
data

array([[[[170, 165, 143],
         [189, 184, 162],
         [171, 166, 144],
         ..., 
         [152, 147, 125],
         [132, 126, 104],
         [139, 133, 111]],

        [[148, 143, 121],
         [128, 123, 101],
         [100,  95,  73],
         ..., 
         [154, 149, 127],
         [127, 122, 100],
         [141, 135, 113]],

        [[139, 134, 112],
         [116, 111,  89],
         [120, 115,  93],
         ..., 
         [148, 143, 121],
         [124, 119,  97],
         [139, 133, 111]],

        ..., 
        [[134, 132, 109],
         [141, 136, 114],
         [149, 145, 120],
         ..., 
         [159, 154, 132],
         [158, 153, 131],
         [143, 138, 116]],

        [[138, 136, 113],
         [133, 129, 104],
         [150, 146, 121],
         ..., 
         [150, 145, 123],
         [128, 123, 101],
         [126, 121,  99]],

        [[136, 134, 111],
         [138, 136, 111],
         [139, 135, 110],
         ..., 
         [134, 129, 107],
  